# Exercise: Performance Optimization 1

Optimize the following code, that is, try to reduce the runtime as well as the number of allocations as much as you can.

In [1]:
function work!(A, b, c)
    D = zeros(N,N)
    for i in 1:N
        D = b[i]*c*A
        b[i] = sum(D)
    end
    return b
end

work! (generic function with 1 method)

The following data is **fixed** and **not supposed to be modified**!

In [2]:
using Random
Random.seed!(42)

N = 1000
A = rand(N,N)
b = rand(N)
c = 1.23

const b_result = work!(A, b, c);

You can compare against `b_result` to test your implementation(s):

In [3]:
using Test 

@test work!(A, b, c) ≈ b_result

Test Passed

You can benchmark as follows:

In [17]:
using BenchmarkTools

@benchmark work!($A, $b, $c)

BenchmarkTools.Trial: 6 samples with 1 evaluation.
 Range (min … max):  868.974 ms … 873.410 ms  ┊ GC (min … max): 7.22% … 7.29%
 Time  (median):     870.834 ms               ┊ GC (median):    7.28%
 Time  (mean ± σ):   871.231 ms ±   1.752 ms  ┊ GC (mean ± σ):  7.28% ± 0.05%

  █              █        █ █                               █ █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁█▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁█ ▁
  869 ms           Histogram: frequency by time          873 ms <

 Memory estimate: 7.46 GiB, allocs estimate: 7981.

## Your Optimizations

Your optimized variants go here!

**Hints** (hopefully):
* Is the function self-contained?
* Is it efficient with respect to allocations?
* An O(10000) speedup and 0 allocations are possible on Noctua 2 😉

### Avoiding globals

In [5]:
# @code_warntype work!(A, b, c)

In [18]:
function work1!(A, N, b, c) # N is now a function argument
    D = zeros(N,N)
    for i in 1:N
        D = b[i] * c * A
        b[i] = sum(D)
    end
    return b
end

@test work1!(A, N, b, c) ≈ b_result

Test Passed

In [19]:
@benchmark work1!($A, $N, $b, $c)

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  830.030 ms … 840.502 ms  ┊ GC (min … max): 6.52% … 6.77%
 Time  (median):     831.774 ms               ┊ GC (median):    6.53%
 Time  (mean ± σ):   833.902 ms ±   4.085 ms  ┊ GC (mean ± σ):  6.62% ± 0.19%

  █  ██     █                           █ █                   █  
  █▁▁██▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  830 ms           Histogram: frequency by time          841 ms <

 Memory estimate: 7.46 GiB, allocs estimate: 2002.

In [7]:
# @code_warntype work1!(A, N, b, c)

### Avoid (some) temporary allocations

In [20]:
function work2!(A, N, b, c)
    D = zeros(N,N)
    for i in 1:N
        @. D = b[i] * c * A
        b[i] = sum(D)
    end
    return b
end

@test work2!(A, N, b, c) ≈ b_result

Test Passed

In [21]:
@benchmark work2!($A, $N, $b, $c)

BenchmarkTools.Trial: 18 samples with 1 evaluation.
 Range (min … max):  279.755 ms … 290.253 ms  ┊ GC (min … max): 0.00% … 0.07%
 Time  (median):     280.543 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   281.689 ms ±   2.792 ms  ┊ GC (mean ± σ):  0.01% ± 0.03%

  ▃ ▃▃█▃                                                         
  █▇████▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▁▁▇▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇ ▁
  280 ms           Histogram: frequency by time          290 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

Alternatively without broadcasting but a loop:

In [22]:
function work3!(A, N, b, c)
    D = zeros(N,N)
    for i in 1:N
        for j in eachindex(D)
            @inbounds D[j] = b[i] * c * A[j]
        end
        b[i] = sum(D)
    end
    return b
end

@test work3!(A, N, b, c) ≈ b_result

Test Passed

In [23]:
@benchmark work3!($A, $N, $b, $c)

BenchmarkTools.Trial: 18 samples with 1 evaluation.
 Range (min … max):  279.156 ms … 290.298 ms  ┊ GC (min … max): 0.00% … 0.06%
 Time  (median):     280.244 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   281.485 ms ±   3.033 ms  ┊ GC (mean ± σ):  0.01% ± 0.03%

     █ ▁▁ ▁                                                      
  ▆▁▆█▆██▁█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  279 ms           Histogram: frequency by time          290 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

### Preallocating `D`

In [24]:
function work4!(A, N, b, c, D)
    for i in 1:N
        @. D = b[i] * c * A
        b[i] = sum(D)
    end
    return b
end

D = zeros(N,N)

@test work4!(A, N, b, c, D) ≈ b_result

Test Passed

In [25]:
@benchmark work4!($A, $N, $b, $c, $D)

BenchmarkTools.Trial: 18 samples with 1 evaluation.
 Range (min … max):  279.221 ms … 283.481 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     279.525 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   279.932 ms ±   1.068 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █  ▄▄                                                          
  █▁▆██▆▁▆▁▁▁▆▆▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  279 ms           Histogram: frequency by time          283 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

### Realizing that one can factor out `b` and `c`

In [26]:
# function work5!(A, N, b, c; D)
#     for i in 1:N
#         @. D = b[i] * c * A
#         b[i] = sum(D)
#     end
#     return b
# end

# function work5!(A, N, b, c; D)
#     for i in 1:N
#         b[i] = sum(b[i] * c * A)
#     end
#     return b
# end

# function work5!(A, N, b, c; D)
#     for i in 1:N
#         b[i] = b[i] * c * sum(A)
#     end
#     return b
# end

function work5!(A, N, b, c)
    b .*= c * sum(A)
    return b
end

@test work5!(A, N, b, c) ≈ b_result

Test Passed

In [27]:
@benchmark work5!($A, $N, $b, $c)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  98.350 μs … 306.011 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     99.471 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   99.789 μs ±   3.401 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▁▂▄▅▆▇▇████▇▇▆▅▃▃▂▁▁▁        ▁▁▁▂▂▂▁        ▁▁▁▁▁▁▁ ▁   ▃
  ▃▂▅▅▇█████████████████████▇▇▇▆▅▅▇▇█████████▇▇▇▇███████████▇▇ █
  98.4 μs       Histogram: log(frequency) by time       103 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.